In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

font = {'size': 22}

matplotlib.rc('font', **font)

import numpy as np
import pandas as pd

import scipy
import scipy.stats
import seaborn as sns

import sklearn.datasets

import mrob

# Pose compounding

In [3]:
T_1 = mrob.geometry.SE3([0,0,0,1,0,0])
sigma_1 = np.diag([0,0,0.03**2,0,0,0])
T_1.T()

array([[1., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [4]:
T_2 = mrob.geometry.SE3([0,0,0,0,1,0])
sigma_2 = np.diag([0,0,0.03**2,0,0,0])
T_2.T()

array([[1., 0., 0., 0.],
       [0., 1., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## Second order

In [5]:
T = T_1.mul(T_2)

In [6]:
T_1_adj = T_1.adj()
T_1_adj

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.]])

In [7]:
xi_2_ = T_1_adj @ xi_2
xi_2_

NameError: name 'xi_2' is not defined

In [ ]:
sigma_2_ = T_1_adj@sigma_2@T_1_adj.transpose()

In [ ]:
sigma = sigma_1 + sigma_2_

In [ ]:
def compound_2nd(T1, sigma1, T2, sigma2):
    T = T_1.mul(T_2)
    T_1_adj = T_1.adj()
    sigma_2_ = T_1_adj@sigma_2@T_1_adj.transpose()
    sigma = sigma_1 + sigma_2_
    return T, sigma

In [ ]:
T, sigma = compound_2nd(T_1, sigma_2,T_2, sigma_2)

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(sigma,annot=True)

## Cholesky for block matrix

In [ ]:
def cholesky(sigma):
    condition =~ (np.all(sigma == 0, axis=1) & (np.all(sigma == 0, axis=0)))
    m = [int(x) for x in condition]
    counter = 0
    res = []
    for el in m:
        if el > 0:
            res.append(counter)
            counter +=1
        else:
            res.append(None)
    M = []    
    for i in range(6):
        tmp = []
        for j in range(6):
            tmp.append([res[i],res[j]])
        
        M.append(tmp)
    M = np.array(M)
    
    block = (sigma[condition,:])[:,condition]

    L = np.linalg.cholesky(block)
    LL = np.zeros_like(sigma)
    
    for i in range(LL.shape[0]):
        for j in range(LL.shape[1]):
            if all(M[i,j] != None):
                k = M[i,j][0]
                l = M[i,j][1]
            
                LL[i,j] = L[k,l]
        
    return LL

In [ ]:
A = cholesky(sigma)
print(A)

In [ ]:
np.linalg.norm(sigma - A@A.transpose())

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(A,annot=True)

In [ ]:
N = 100
error = 0
for _ in range(N):
    tmp = np.random.randn(6)+5
    tmp[np.random.randint(0,6)] = 0
    sigma = np.diag(tmp)
    L = cholesky(sigma)
    error += np.linalg.norm(sigma - L@L.transpose())

print("Mean error: {}".format(error/N))

# Linear operators

In [ ]:
def op1(A):
    return -np.eye(A.shape[0])*A.trace() + A

def op2(A,B):
    return op1(A)@op1(B) + op1(B@A)

# Fourth order

In [ ]:
def compound_4th(T_1, sigma_1, T_2, sigma_2):
    T, sigma = compound_2nd(T_1, sigma_1, T_2, sigma_2)
    
    sigma_1_rho_rho = sigma_1[3:, 3:]
    sigma_1_rho_phi = sigma_1[3:, :3]
    sigma_1_phi_phi = sigma_1[:3, :3]
    
    T_1_adj = T_1.adj()
    
    _sigma_2 = T_1_adj @ sigma_2 @ T_1_adj.transpose()
    
    _sigma_2_rho_rho = _sigma_2[3:, 3:]
    _sigma_2_rho_phi = _sigma_2[3:, :3]
    _sigma_2_phi_phi = _sigma_2[:3, :3]
    
    A_1 = np.zeros((6,6))
    A_1[:3, :3] = op1(sigma_1_phi_phi)
    A_1[:3, 3:] = op1(sigma_1_rho_phi + sigma_1_rho_phi.transpose())
    A_1[3:, 3:] = op1(sigma_1_phi_phi)
    
    _A_2 = np.zeros((6,6))
    _A_2[:3, :3] = op1(_sigma_2_rho_rho)
    _A_2[:3, 3:] = op1(_sigma_2_rho_phi + _sigma_2_rho_phi.transpose())
    _A_2[3:, 3:] = op1(_sigma_2_phi_phi)
    
    B_rho_rho = op2(sigma_1_phi_phi, _sigma_2_rho_rho) + \
        op2(sigma_1_rho_phi.transpose(), _sigma_2_rho_phi) + \
        op2(sigma_1_rho_phi,_sigma_2_rho_phi.transpose()) + \
        op2(sigma_1_rho_rho, _sigma_2_phi_phi)
    
    B_rho_phi = op2(sigma_1_phi_phi,_sigma_2_rho_phi.transpose()) + \
        op2(sigma_1_rho_phi.transpose(),_sigma_2_phi_phi)
    
    B_phi_phi = op2(sigma_1_phi_phi, _sigma_2_phi_phi)
    
    B = np.array([[B_phi_phi, B_rho_phi],[B_rho_phi.transpose(), B_rho_rho]]).reshape((6,6))
    
    sigma += 1/12*(A_1@_sigma_2 + \
                   _sigma_2@A_1.transpose() + \
                   _A_2@sigma_1 + \
                   sigma_1@ _A_2.transpose()) + 0.25*B
    
    return T, sigma
    

In [ ]:
T, sigma = compound_4th(T_1, sigma_1, T_2, sigma_2)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(sigma,annot=True)